# Conformer Search Analysis

After running `scripts/3_compute.py` analyze your results here!

In [58]:
from pathlib import Path

from qcio import Structure, view, ProgramOutput, align, rmsd, constants

In [59]:
# Constants
DATA_DIR = Path("../data")
STRUCT_DIR = DATA_DIR / "structures"
CALC_DIR = DATA_DIR / "calcs"

## Open and Inspect Results

We are using the same pattern here that we used for `Structure` objects and the `energy/optimization` calculations: open a calculation with `ProgramOuput.open("path/to/file.json")` and then passed to `view.view(...)` to get a visualization.

**Notice that some structures look very similar and are very similar in energy. Is CREST really returning unique conformers??**

In [14]:
u2_conf = ProgramOutput.open(CALC_DIR / "u2-anion-crest-conformer_search.json")

In [15]:
view.view(u2_conf)

## Conformer Analysis
Conformers 0,1,2 look very similar! So do 3,4,5. Let's see if we can figure out what's going on!

**See below that the structures look almost identical but with the -CF<sub>3</sub> groups having different numbering. CREST is clearly not considering symmetry when determining the RMSD for structures!**

In [32]:
view.view(u2_conf.results.conformers[0], u2_conf.results.conformers[1], show_indices=True)

### Use qcio's rmsd/align algorithms to quantify similarity and include symmetry

In [52]:
# rmsd aligns structures and considers symmetry by default. To ignore symmetry pass `symmetry=False` (runs faster)
rmsd_bohr = rmsd(u2_conf.results.conformers[0], u2_conf.results.conformers[1])
print(f"RMSD in Bohr: {rmsd_bohr:.3f}")
print(f"RMSD in Angstrom: {rmsd_bohr * constants.BOHR_TO_ANGSTROM:.3f}")

RMSD in Bohr: 0.666
RMSD in Angstrom: 0.352


#### Align structures to get a visual of what this RMSD looks like

In [54]:
# Align conf0 onto conf1 and consider symmetry. algin(this_struc, onto_this_struct)
conf1_aligned0, rmsd_ang = align(u2_conf.results.conformers[1], u2_conf.results.conformers[0], length_unit='angstrom')
print(f"RMSD in Angstrom: {rmsd_ang:.3f}")

RMSD in Angstrom: 0.352


### Clearly the structures are nearly identical! Conf_1 is just a -CF<sub>3</sub> rotation of Conf_0

In [55]:
view.view(u2_conf.results.conformers[0], conf1_aligned0, same_viewer=True)

### Now that we know how to do manual conformers analysis, let's use qcio to automatically perform filtering for us taking symmetry into account

We will use a `confomer_rmsd_threshold` of `1.0` Bohr (`0.53` Angstroms).

Looke like there are 4 unique conformers for this structure and that 0,1 and 2,3 are rotational conformers about the N-C bond to the aryl wing!

In [57]:
view.view(u2_conf, conformer_rmsd_threshold=1.0)

## More detailed analysis

In [60]:
filtered_conf, filtered_energies = u2_conf.results.conformers_filtered(threshold=1.0)

### RMSD On all Structures
These look pretty unique! 0,3 are most similar, let's dive in more!

In [66]:
for i in range(len(filterd_confs)):
    for j in range(i + 1, len(filterd_confs)):
        print(f"RMSD Conf ({i}, {j}): {rmsd(filterd_confs[i], filterd_confs[j]):.3f} Bohr")
    

RMSD Conf (0, 1): 3.416 Bohr
RMSD Conf (0, 2): 4.454 Bohr
RMSD Conf (0, 3): 1.846 Bohr
RMSD Conf (1, 2): 2.083 Bohr
RMSD Conf (1, 3): 4.148 Bohr
RMSD Conf (2, 3): 4.331 Bohr


In [69]:
fconf3_alinged0, rmsd_val3_0 = align(filterd_confs[3], filterd_confs[0])
print(f"RMSD: {rmsd_val3_0:.3f} Bohr")

RMSD: 1.846 Bohr


#### qcio makes it easy to see how structures are similar/different

In [71]:
view.view(filterd_confs[0], fconf3_alinged0, same_viewer=True)